In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load the dataset
df = pd.read_csv('dataset_sdn.csv')

# Drop unnecessary columns (if any)
df = df.drop(columns=['dt', 'switch', 'src', 'dst', 'Protocol', 'port_no'])

# Encode categorical columns (if any)
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Split features and labels
X = df.drop(columns=['label'])
y = df['label']

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape data for LSTM (samples, timesteps, features)
# Assuming each row is a timestep and we have 1 timestep per sample
X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(X_reshaped.shape[1], X_reshaped.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # Use 'softmax' for multi-class classification

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model on the full dataset
history = model.fit(X_reshaped, y, epochs=20, batch_size=32, verbose=1)

# Save the model
model.save('lstm_sdn_model_full.h5')

# Function to preprocess user input and make predictions
def predict_user_input(model, scaler, input_data):
    # Convert input data to a DataFrame
    input_df = pd.DataFrame([input_data], columns=X.columns)
    
    # Normalize the input data using the same scaler
    input_scaled = scaler.transform(input_df)
    
    # Reshape the input data for LSTM (samples, timesteps, features)
    input_reshaped = input_scaled.reshape((input_scaled.shape[0], 1, input_scaled.shape[1]))
    
    # Make predictions
    prediction = model.predict(input_reshaped)
    prediction = (prediction > 0.5).astype(int)  # Convert probabilities to binary predictions
    
    # Decode the prediction (if label encoder was used)
    if label_encoder:
        prediction = label_encoder.inverse_transform(prediction)
    
    return prediction[0]

# Example user input (replace this with actual input)
# Ensure the input matches the features used during training
user_input = {
    'pktcount': 45304,
    'bytecount': 48294064,
    'dur': 100,
    'dur_nsec': 716000000,
    'tot_dur': 1.01e+11,
    'flows': 3,
    'packetins': 1943,
    'pktperflow': 13535,
    'byteperflow': 14428310,
    'pktrate': 451,
    'Pairflow': 0,
    'tx_bytes': 143928631,
    'rx_bytes': 3917,
    'tx_kbps': 0,
    'rx_kbps': 0,
    'tot_kbps': 0
}

# Make prediction
prediction = predict_user_input(model, scaler, user_input)
print(f"Prediction: {prediction}")

C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 1, 50)               │          13,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1, 50)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 33,651 (131.45 KB)

 Trainable params: 33,651 (131.45 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
3261/3261 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.6068 - loss: nan
Epoch 2/20
3261/3261 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6075 - loss: nan
Epoch 3/20
3261/3261 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6102 - loss: nan
Epoch 4/20
3261/3261 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6100 - loss: nan
Epoch 5/20
3261/3261 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6107 - loss: nan
Epoch 6/20
3261/3261 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6064 - loss: nan
Epoch 7/20
3261/3261 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6085 - loss: nan
Epoch 8/20
3261/3261 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6064 - loss: nan
Epoch 9/20
3261/3261 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6085 - loss: nan
Epoch 10/20
3261/3261 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6076 - loss: nan
Epoch 11/20
3261/3261 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6088 - loss: nan
Epoch 12/20
3261/3261 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accura

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
Prediction: 0


C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
